In [1]:
import madge from 'npm:madge';

const dependencies = await madge(Deno.cwd() + '/opencrvs-core/packages/client/src/index.tsx', {
  tsConfig: Deno.cwd() + '/opencrvs-core/packages/client/tsconfig.json',
  baseDir: Deno.cwd() + '/opencrvs-core',
}).then((res) => res.obj())


In [131]:
async function getAllFiles() {
  const prUrl =
    "https://api.github.com/repos/opencrvs/opencrvs-core/pulls/7301";

  const headers = {
    Accept: "application/vnd.github.v3+json",
  };

  let allFiles = [];
  let page = 1;
  let hasMore = true;

  while (hasMore) {
    const response = await fetch(`${prUrl}/files?per_page=100&page=${page}`, {
      headers,
    });

    if (!response.ok) {
      throw new Error(`GitHub API error: ${response.status} ${response.statusText}`);
    }

    const files = await response.json();
    allFiles = allFiles.concat(files);

    if (files.length < 100) {
      hasMore = false; // No more pages
    } else {
      page += 1;
    }
  }

  return allFiles;
}

In [133]:
const files = await getAllFiles()

In [3]:
import { toStream } from 'npm:ts-graphviz/adapter';

async function dotToSVG(dot: string): string {
  const stream = await toStream(dot, { format: 'svg' });
  const reader = stream.getReader();
  const decoder = new TextDecoder();
  let output = "";

  while (true) {
    const { done, value } = await reader.read();
    if (done) break;
    output += decoder.decode(value, { stream: true });
  }
  return output
}


In [ ]:
import { posix } from "https://deno.land/std@0.207.0/path/mod.ts";
type File = {
  filename: string;
};

const filesInThePR: File = files;

function fileToNode(file: File) {
  const nodeId = file.filename.replace(/[^a-zA-Z0-9]/g, "_");
  return new Node(nodeId, {
    label: posix.basename(file.filename),
    style: "filled",
    color: "#ADD8E670",
  });
}

function filesToDot(files: File[]): Digraph {
  const graph = new Digraph("", { rankdir: "LR" });

  for (const file of files) {
    graph.addNode(fileToNode(file));
  }

  return graph;
}

function addParentDirectories(files: File[]) {
  const graph = new Digraph("" /*, { rankdir: "LR" }*/);
  const directories = new Map<string, Node>();

  for (const file of files) {
    const fileNode = fileToNode(file);
    graph.addNode(fileNode);
    const directoryNames = file.filename.split("/").slice(0, -1);
    const fileDirectory = directoryNames.join("/");
    const directoriesPaths = directoryNames.map((dir, i) =>
      directoryNames.slice(0, i + 1).join("/")
    );
    directoriesPaths.forEach((directory) => {
      if (!directories.has(directory)) {
        const node = new Node(directory, {
          label: posix.basename(directory),
          color: "#c6c5fe",
          shape: "box",
          style: "rounded",
          style: "filled",
          edgeColor: '#757575',
        });

        directories.set(directory, node);

        graph.addNode(node);
        const parentPath = directory.split("/").slice(0, -1).join("/");
        const parentNode = directories.get(parentPath);

        if (parentNode) {
          graph.createEdge([parentNode, node]);
        }
      }
    });

    graph.createEdge([directories.get(fileDirectory), fileNode]);
  }

  return graph;
}

const graph = addParentDirectories(filesInThePR, filesToDot(filesInThePR));
const dot = toDot(graph);

Deno.jupyter.svg([await dotToSVG(dot)]);


In [ ]:
import { join } from "https://deno.land/std/path/mod.ts";
import { Digraph, Graph, Node, Edge, toDot } from "npm:ts-graphviz";
import { posix } from "https://deno.land/std@0.207.0/path/mod.ts";



function isInFiles(path: string): boolean {
  const projectPath = path.replace(Deno.cwd() + "/opencrvs-core/", "");
  return files.some((file) => file.filename.startsWith(projectPath));
}

function statusColor(status: "modified") {
  switch (status) {
    case "modified":
      return "#d7d467";
    case "added":
      return "#67d78b";
    default:
      return "ADD8E610";
  }
}

async function getRelevantEntries(
  path: string
): Promise<
  { path: string; name: string; isDirectory: boolean; status?: string }[]
> {
  const entries: {
    path: string;
    name: string;
    isDirectory: boolean;
    status?: string;
  }[] = [];

  async function traverseDirectory(path: string) {
    for await (const entry of Deno.readDir(path)) {
      const currentPath = join(path, entry.name);

      if (!isInFiles(currentPath)) {
        continue;
      }

      const projectPath = currentPath.replace(
        Deno.cwd() + "/opencrvs-core/",
        ""
      );
      const file = files.find((file) => file.filename === projectPath);

      entries.push({
        path: toProjectPath(currentPath),
        name: entry.name,
        isDirectory: entry.isDirectory,
        status: file?.status,
      });

      if (entry.isDirectory) {
        await traverseDirectory(currentPath);
      }
    }
  }

  await traverseDirectory(path);
  return entries;
}

function toProjectPath(path: string): string {
  return path.replace(Deno.cwd() + "/opencrvs-core/", "");
}
const allDeps = Object.entries(dependencies)
function renderGraph(
  entries: {
    path: string;
    name: string;
    isDirectory: boolean;
    status?: string;
  }[],
  graphName: string = "DirectoryGraph"
) {
  const graph = new Digraph(graphName, {rankdir: "LR"});
  const nodes: Record<string, Node> = {};

  entries.forEach((entry) => {
    const nodeId = entry.path.replace(/[^a-zA-Z0-9]/g, "_");
    const currentNode = graph.createNode(nodeId, {
      label: entry.name,
      style: "filled",
      color: entry.isDirectory ? "#ADD8E670" : statusColor(entry.status),
    });

    nodes[entry.path] = currentNode;
  });

  entries.forEach((entry) => {
    const parentPath = entry.path.substring(0, entry.path.lastIndexOf("/"));
    if (nodes[parentPath]) {
      graph.createEdge([nodes[parentPath], nodes[entry.path]]);
    }
  });

  entries.forEach((entry) => {
    const projectPath = entry.path;
    const deps = allDeps.filter(([main, deps]) => deps.includes(projectPath)).map(([main]) => main);
    if (!entry.isDirectory && deps) {
      deps.forEach((dep: string) => {
        const depPath = dep;
        const depNode = nodes[depPath];

        if (depNode) {
          graph.createEdge([depNode, nodes[projectPath]], { style: "dotted" });
        }
      });
    }
  });

  entries.forEach((entry) => {
    const deps = allDeps.filter(([main, deps]) => deps.includes(entry.path)).map(([main]) => main);

    if (deps) {
      deps.forEach((dep: string) => {
        let depPath = dep;
        let depNode = nodes[depPath];

        if (!depNode) {

          const nodeId = depPath.replace(/[^a-zA-Z0-9]/g, "_");
          const currentNode = graph.createNode(nodeId, {
            label: posix.basename(depPath),
            style: "filled",
            color: "#cccccc50"
          });

          nodes[depPath] = currentNode;
          depNode = currentNode;

          let parentPath = depPath.substring(0, depPath.lastIndexOf("/"));
          let parentExists = false
          while(!parentExists) {
            if (!nodes[parentPath]) {
              const nodeId = parentPath.replace(/[^a-zA-Z0-9]/g, "_");
              const currentNode = graph.createNode(nodeId, {
                label: posix.basename(parentPath),
                style: "filled",
                color: "#ADD8E620"
              });

              nodes[parentPath] = currentNode;
            } else {


              parentExists = true
            }
            graph.createEdge([nodes[parentPath], nodes[depPath]]);
            depPath = parentPath;
            parentPath = parentPath.substring(0, parentPath.lastIndexOf("/"));
          }
        }

        graph.createEdge([depNode, nodes[entry.path]], { style: "dotted" });
      });
    }
  });

  return graph;
}

const dirPath = Deno.cwd() + "/opencrvs-core/packages/client";
const entries = await getRelevantEntries(dirPath);

const graph = renderGraph(entries);
Deno.jupyter.svg([await dotToSVG(toDot(graph))]);
